<a href="https://colab.research.google.com/github/XekoFrontend/google-colab-notebook/blob/main/rclone_gui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📌 **Install**


In [ ]:
# Cài đặt rClone
!sudo -v ; curl https://rclone.org/install.sh | sudo bash

In [ ]:
# Danh sách remotes đang có
!rclone listremotes

In [ ]:
# Khôi phục file config
!cp "/content/drive/MyDrive/Colab Notebooks/rclone.conf" /root/.config/rclone/rclone.conf

# ⚙️ **Config remotes**

Modify remotes

In [ ]:
# Tạo mới, chỉnh sửa, xóa hoặc kiểm tra remote.
!rclone config

Backup config

In [ ]:
# Backup config
!cp /root/.config/rclone/rclone.conf /content/rclone.conf
!rclone copy -v "/content/rclone.conf" "/content/drive/MyDrive/Colab Notebooks/backup/rclone-renamed.conf"

# 🎯 **Các lệnh Copy**

### Option flag

**Command**
* `copy` : Copy file/thư mục từ nguồn tới đích, bỏ qua file giống nhau
* `check` : So sánh nội dung giữa nguồn và đích
* `sync` : đồng bộ (dùng --dry-run kiểm tra trước khi chạy thật).
* `ls`: Liệt kê file và kích thước (mặc định).
* `lsf`: Chỉ hiển thị tên file/thư mục.
* `lsd`: Chỉ hiển thị tên thư mục con.
* `size`: Hiển thị tổng kích thước của thư mục.

---
**Flag**

* Ghi đè nếu nguồn mới hơn: `--update`. (không dùng được trên archive)
* Bỏ qua các file đã tồn tại: `--ignore-existing`. (phù hợp với *archive.org*, còn lại thì không cần, dùng `copy` là đủ).
* Chọn số luồng copy (nếu không dùng thì mặc định là 4)): `--transfers 8`.
* Dùng cho S3 của Amazon: `--s3-upload-concurrency 1`. (archive.org tối đa dùng được 2 luồng, file lớn ưu tiên dùng 1 luồng).

* `--dry-run` : Chạy thử, kiểm tra trước.

* `--include "*.mp4"` để chỉ copy file .mp4.
* `--exclude "*.tmp`" để bỏ qua file tạm.
 * Ex lọc files: `!rclone ls "archive:cd-luong-bich-huu-co-gai-trung-hoa" --include "*.flac"`
 * Ex 2: `rclone copy source dest --include "*.jpg" --exclude "*.gif"`

In [ ]:
# @title Basic: Copy files between Cloud Remotes

source_remote = "googledrive1" # @param ["/content/drive/MyDrive/","/content/","googledrive1","onedrive","googledrive2","archive"]
input_folder = "HienRau-Guitar Đệm Hát Trong 30 Ngày/" # @param {"type":"string","placeholder":"dùng `/` để phân cấp folder hoặc file, để trống là toàn bộ remote"}
# @markdown ###Lưu ý:
# @markdown - nếu thêm `/` ở cuối, chỉ copy nội dung bên trong
# @markdown - không dùng `/` ở cuối thì copy cả folder
# @markdown ---

destination_remote = "archive" # @param ["archive","onedrive","googledrive2","googledrive1","/content/drive/MyDrive/","/content/"]
destination_folder = "HienRau-Guitardemhat30Ngay" # @param {"type":"string","placeholder":"dùng `/` để phân cấp folder"}

# @markdown ---
# @markdown ### Hướng dẫn:
# @markdown - Dùng khoảng trắng để phân cách các flag.

option_flag = "--s3-upload-concurrency 2" # @param {"type":"string","placeholder":"thêm `--flag` nếu cần, dùng space phân cách các flag"}

# Kiểm tra và thực thi lệnh copy
if source_remote and destination_remote:
    # Xử lý đường dẫn
    source_path = f"{source_remote}:{input_folder}" if input_folder else f"{source_remote}:"
    dest_path = f"{destination_remote}:{destination_folder}" if destination_folder else f"{destination_remote}:"

    # Xử lý option_flag
    if option_flag.strip():
        flags = option_flag.strip().split()
        command_flags = "-v " + " ".join(flags)  # Kết hợp -v với các flag người dùng nhập
    else:
        command_flags = "-v"  # Mặc định chỉ dùng -v

    # Chạy lệnh rclone
    !rclone copy {command_flags} "{source_path}" "{dest_path}"
else:
    print("Lỗi: Vui lòng nhập cả Source Remote và Destination Remote.")


	- Lưu kết quả:
		○  > sync_result.txt
	- Báo cáo files lỗi:
		○ --error string
		○ --stats-log-level INFO
* `--log-level LogLevel                  Log level DEBUG|INFO|NOTICE|ERROR (default NOTICE)`
- Xóa files : delete -i
-Xóa toàn bộ folder: purge -i




* Thêm `--low-level-retries 10` để thử lại khi gặp lỗi mạng.

# 👓 **Xem Nội Dung Remote**

In [ ]:
# @markdown ### Hướng dẫn:
# @markdown - **List Mode**: Chọn cách hiển thị nội dung:
# @markdown   - `ls`: Liệt kê file và kích thước (mặc định).
# @markdown   - `lsf`: Chỉ hiển thị tên file/thư mục.
# @markdown   - `lsd`: Chỉ hiển thị tên thư mục con.
# @markdown   - `size`: Hiển thị tổng kích thước của thư mục.

list_remote = "/content/drive/MyDrive/" # @param ["/content/drive/MyDrive/","/content/","googledrive1","onedrive","googledrive2","archive"]
input_folder = "tinhhe-lontien-thyetmin/" # @param {"type":"string"}
list_mode = "lsf" # @param ["lsf","lsd","ls","size"]
# @markdown 🔎 Lọc file:
file_type = "mkv" # @param {"type":"string"}
file_type_sum = "Default" # @param ["Default", "Sum"]

# Xử lý file_type_sum thành lệnh phù hợp
sum_command = "" if file_type_sum == "Default" else " | wc -l"


!rclone {list_mode} "{list_remote}:{input_folder}"  | grep ".{file_type}"  {sum_command}


# ======================================
# grep ".mp4": Lọc các dòng có chứa .mp4.
# wc -l: Đếm số dòng (tức là số file .mp4).

# 🗃️ **.Rar .Zip .7zip**

## Rar

In [ ]:
# Install unrar
!apt-get install unrar

In [ ]:
# Giải nén với password (ex: DDDTT)
!unrar x -pdomdom2010 /content/copy_temp/*.zip /content/copy_temp/extracted/

## Zip

In [ ]:
# Install unzip
!apt-get install -y unzip

In [ ]:
# Giải nén file .zip với password (nếu có)
!unzip -P kirakira /content/copy_temp/Ed01.zip -d /content/copy_temp/extracted/

In [ ]:
# Dùng vòng lặp `for` Giải nén nhiều file .zip trong thư mục với password (nếu có)
import os
for zip_file in os.listdir("/content/copy_temp/2/"):
    if zip_file.endswith(".zip"):
        !unzip -P domdom2010 "/content/copy_temp/2/{zip_file}" -d /content/copy_temp/2/s2/
        print(f"Đã giải nén {zip_file}")

---
## uncheck

rar

In [ ]:
# hình như là nén files, hoặc ghép files. cần thử lại.
!cat /content/torrent_download/HI1\ DDDTT.part*.rar > /content/HI1_DDDTT_full.rar

In [ ]:
# Split file nén
!split -b 10G /content/HI1_DDDTT_full.rar /content/HI1_DDDTT_part_
# copy file trùng tên, chhia theo thứ tự.
!rclone copy /content/HI1_DDDTT_part_* archive:tinhhe-lontien-thyetmin/1990/ -v

7zip
* p7zip-full hỗ trợ hầu hết định dạng (.7z, .zip, .rar, .iso, .tar, .gzip, v.v.), nhưng hiệu suất giải nén .rar tốt hơn với unrar.

In [ ]:
!apt-get update
!apt-get install -y p7zip-full

In [ ]:
!7z x /content/file.7z -o/content/extracted/ -pYOUR_PASSWORD

In [ ]:
!7z x /content/file.iso -o/content/extracted/

In [ ]:
import os

# Đường dẫn thư mục chứa file nén
source_folder = "/content/copy_temp/2/"
extract_folder = "/content/copy_temp/2/zip2/"
password = "domdom2010"  # Thay bằng password thực tế

# Tạo thư mục đích nếu chưa có
!mkdir -p {extract_folder}

# Lặp qua các file trong thư mục
for filename in os.listdir(source_folder):
    if filename.endswith(".zip"):
        file_path = os.path.join(source_folder, filename)
        # Sử dụng 7z để giải nén
        !7z x "{file_path}" -o"{extract_folder}" -p{password} -y
        if os.path.exists(os.path.join(extract_folder, filename.replace(".zip", ""))):
            print(f"Đã giải nén {filename} thành công")
        else:
            print(f"Lỗi: Không thể giải nén {filename}. Kiểm tra file hoặc password!")

cai đặt cả 3

In [ ]:
# Cài đặt các công cụ giải nén
!apt-get update
!apt-get install -y unrar unzip p7zip-full

# 📺 **yt-dlp**

## ⚙️ Install yt-dlp & ffmpeg
* ⚠️Chú ý dùng ! trước command
* Nếu lỗi 403 Forbidden, thêm --geo-bypass

In [ ]:
pip install -U yt-dlp

In [ ]:
!sudo apt install ffmpeg

## ⏬ Hướng dẫn


*   `!yt-dlp`: không dùng flag thì tự chọn định dạng tốt nhất
*   `-v`: để xem log
*   Subtitle:
 *   `--write-subs --sub-lang en`: tải 1 hoặc nhiều subtitles `en,vi,kr`. (ex: *en = english*)
 *   `--all-subs`: tải file và tất cả subtitles.
 *   `--list-subs`: danh sách các subtitles.
 *   `--skip-download`: bỏ qua không tải files, nếu chỉ tải subtitle.

*   `-F`: xem tất cả định dạng
*   `-f`: chọn tải định dạng (ex: `-f 251`)

*   `--get-urlurl`: tải .m3u8

*   `-o`: đặt tên file.
*   `-P`: path đường dẫn để lưu file.

---
`--postprocessor-args "-ss 00:00 -to 00:55"`: cắt âm thanh <~ chua test


In [ ]:
!yt-dlp --all-subs "https://wetv.vip/vi/play/87t5rpu3kbh1qm8/" -o "Xin_Hay_Yeu_Em_E7" -P "Dame na Watashi ni Koishite Kudasai 2016"